In [1]:
pixnetFolder = {
    'pixnet_dataset/Google Search 關鍵字log/': ['PIXNET_search_log_000000000000.gz', 'PIXNET_search_log_000000000001.gz',
        'PIXNET_search_log_000000000002.gz', 'PIXNET_search_log_000000000003.gz', 'PIXNET_search_log_url_articles.gz'],
    'pixnet_dataset/HotelsCombined API/':  ['Hackathon_Hotels_TW_CN.csv', 'Hackathon_Hotels_TW_EN.csv', 
        'HotelsCombined API -Hackathon.pdf', 'Themes.xlsx'],
    'pixnet_dataset/HotelsCombined 導流 log - PIXNET 瀏覽 logs/': ['HotelsCombined_log.gz', 'HotelsCombined_log_referrer_url_articles.zip'],
    'pixnet_dataset/HotelsCombined 導流 log - 交易紀錄數量資料/': ['HotelsCombined_transaction_record.gz', 'HotelsCombined_Zone_欄位說明.pdf'],
    'pixnet_dataset/使用者瀏覽log/': ['PIXNET_visit_log_000000000000.gz', 'PIXNET_visit_log_000000000001.gz',
        'PIXNET_visit_log_000000000002.gz', 'PIXNET_visit_log_000000000003.gz', 'PIXNET_visit_log_000000000004.gz',
        'PIXNET_visit_log_000000000005.gz', 'PIXNET_visit_log_url_articles.gz'],
    'pixnet_dataset/使用者點擊log/': ['PIXNET_event_log_000000000000.gz', 'PIXNET_event_log_000000000001.gz',
        'PIXNET_event_log_000000000002.gz', 'PIXNET_event_log_000000000003.gz', 'PIXNET_event_log_000000000004.gz',
        'PIXNET_event_log_000000000005.gz', 'PIXNET_event_log_000000000006.gz', 'PIXNET_event_log_url_articles.zip'],
    'pixnet_dataset/文章資料集/': ['articles_182k.gz']
}

In [2]:
from data_extractor import *
import json
import re

In [3]:
with open('poi_data/scenic_spot_with_alias.json', encoding='utf-8-sig') as f:
    scenic_spot = json.load(f)
    scenic_spot = scenic_spot['XML_Head']['Infos']['Info']

In [4]:
from ArticutAPI.ArticutAPI import Articut

In [5]:
loc_name = {data['Name']: data['alias'] for data in scenic_spot}
with open('UserDefinedDictFile.json', 'w', encoding='utf8') as f:
    json.dump(loc_name, f, indent=4, ensure_ascii=False)

In [6]:
try:
    with open('account.info') as f:
        print('Account file exist')
        accountInfoDICT = json.load(f)
    articut = Articut(username=accountInfoDICT["email"], apikey=accountInfoDICT["apikey"])
    print("Using account Quota")
except:
    articut = Articut()
    print("Using free Quota")

Using free Quota


In [7]:
import time
with open('segmentation_log/segmentation_log.json', 'r', encoding='utf8') as f:
    segmentation_log = json.load(f)

def segment(inputSTR):
    if inputSTR in segmentation_log:
        print('Input found in segmentation log')
        return segmentation_log[inputSTR]['result_pos']
    
    time.sleep(5)
    inputDICT = articut.parse(inputSTR, level="lv1", userDefinedDictFILE='UserDefinedDictFile.json')
    if inputDICT["status"] == False:
        print(inputDICT)
        if inputDICT['msg'].startswith('Internal server error'):
            print('Input string:', inputSTR)
        if inputDICT['msg'].endswith('Articut 目前只能處理現代白話中文。'):
            print('Input string:', inputSTR)
        return ''
    
    print("Segmentation status: ", inputDICT["status"])
    for index, sentence in enumerate(inputDICT["result_pos"]):
        if re.search("<.*?>(.*?)</.*?>", sentence) is None:
            inputDICT["result_pos"][index] = "<Other>" + inputDICT["result_pos"][index] + "</Other>"
    result = ''.join(inputDICT["result_pos"])
    inputDICT["result_pos"] = result
    
    segmentation_log[inputSTR] = inputDICT
    print('New Entry created in segmentation log')
    #print('Segmented string:', result)
    return result

In [8]:
try:
    mapping = {}
    for data in scenic_spot:
        activity = set()
        check_key = ['Toldescribe', 'Description', 'Remarks', 'Keyword']
        for key in check_key:
            if str(data[key]) == '':
                continue
            sentence = re.sub('◼️|■|●', '', str(data[key]))
            sentence = segment(sentence)
            sentence = re.finditer('<ACTION_verb>(.*?)</ACTION_verb>', sentence)
            if sentence is not None:
                activity.update(entry.group(1) for entry in sentence)
        mapping[data['Name']] = list(activity)
except Exception as e:
    print('Exception thrown', str(e))
finally:
    with open('segmentation_log/segmentation_log.json', 'w', encoding='utf8') as f:
        json.dump(segmentation_log, f, indent=4, ensure_ascii=False)

Input found in segmentation log
Input found in segmentation log
Input found in segmentation log
Input found in segmentation log
Input found in segmentation log
Input found in segmentation log
Input found in segmentation log
{'status': False, 'msg': "Internal server error. (Your word count balance is not consumed, don't worry. System will reboot in 5min, please try again later.)", 'product': 'https://api.droidtown.co/product/', 'document': 'https://api.droidtown.co/document/'}
Input string: 台南，志鋼，金屬，觀光工廠
{'status': False, 'msg': "Internal server error. (Your word count balance is not consumed, don't worry. System will reboot in 5min, please try again later.)", 'product': 'https://api.droidtown.co/product/', 'document': 'https://api.droidtown.co/document/'}
Input string: 林口酒廠為擁有近百年歷史的台北酒廠(1921)、板橋酒廠(1937)、樹林酒廠(1906)合併而成的新酒廠，於民國76年春正式啟用量產。原三酒廠各具有不同產品與文化特色，合併後發展為北部地區產能最大、設備最現代化的米糧釀造酒生產工廠。 林口酒廠成立以來一直是以生產紹興酒為主，民國85年利用原紹興酒生產設備釀製日式清酒，成功推出「玉泉清酒」，為全國唯一生產日式清酒之酒廠。民國93年為解決清酒粕副產物問題，開發「台酒清酒粕面膜」產品，定位為

{'status': False, 'msg': "Internal server error. (Your word count balance is not consumed, don't worry. System will reboot in 5min, please try again later.)", 'product': 'https://api.droidtown.co/product/', 'document': 'https://api.droidtown.co/document/'}
Input string: 紫坪位在綠島最南方，從附近的步道，可通往海岸邊的潟湖。此處是由珊瑚礁構成的潮池，也是綠島著名的潟湖所在地，有全綠島最完整的潟湖地形以及珊瑚礁植群，更有茂盛的保育類植物水芫花和珍貴的陸寄居蟹，值得大家共同保護。
{'status': False, 'msg': "Internal server error. (Your word count balance is not consumed, don't worry. System will reboot in 5min, please try again later.)", 'product': 'https://api.droidtown.co/product/', 'document': 'https://api.droidtown.co/document/'}
Input string: 1、紫坪上方的綠島露營區為生態保護區，禁止採集花木生物，並請維護環境整潔，讓這片美景能留與後代子孫。2、露營區目前已於2009年委由「東方之泉有限股份公司」經營，      聯絡電...
Input found in segmentation log
Input found in segmentation log
{'status': False, 'msg': "Internal server error. (Your word count balance is not consumed, don't worry. System will reboot in 5min, please try again later.)", 'product': 'https://api.droidtown.co

In [9]:
with open('activity/POI2activity.json', 'w', encoding='utf8') as f:
    json.dump(mapping, f, indent=4, ensure_ascii=False)